# Agentic AI

## Prerequisites

### Install a Local LLM with Ollama

To run this project locally, we will install and use **Ollama**, a lightweight runtime for local large language models.

**Download Ollama:**  
https://ollama.com/

Once installed, you can pull any model you want to run.  
Below are a few recommended examples, but you are free to pick any size or model from the Ollama library.

ollama pull qwen3:0.6b

or

ollama pull ibm/granite4:350m

or

Choose any model you prefer, make sure the model supports tools.
Browse available models here:
https://ollama.com/library



### Python requirements

In [1]:
#!pip install langgraph langchain-google-genai langchain-core mcp langchain-ollama

## 1. Define FastMCP Tools

In [2]:
from heuristics import *
from color_blocks_state import *
from search import *
import asyncio
from langchain_core.tools import StructuredTool

In [3]:
import re
from typing import Any

def normalize_start_blocks_any(x: Any) -> str:
    """
    Returns canonical "(a,b),(c,d),..." string.
    Accepts:
      - str: "(5,2),(1,3)..."
      - list[str] chunks
      - list[tuple[int,int]] or tuple[tuple[int,int],...]
    """
    # tuple/list of pairs -> convert directly
    if isinstance(x, (list, tuple)) and x and all(isinstance(t, (list, tuple)) and len(t) == 2 for t in x):
        pairs = [f"({int(a)},{int(b)})" for a, b in x]
        return ",".join(pairs)

    # list of strings -> join
    if isinstance(x, list):
        s = ",".join(str(p) for p in x)
    else:
        s = str(x)

    s = s.strip().lstrip("/").strip().strip('"').strip("'")

    # extract (d,d) pairs
    pairs = re.findall(r"\(\s*\d+\s*,\s*\d+\s*\)", s)
    if not pairs:
        raise ValueError(f"start_blocks invalid: expected '(a,b),(c,d),...' got {x!r}")
    return ",".join(p.replace(" ", "") for p in pairs)


def normalize_goal_blocks_any(x: Any) -> str:
    """
    Normalize goal_blocks into canonical "a,b,c,..." string.

    Accepts artifacts like:
      - "/10,2,3,4"
      - "/2,/4,/6,/8"   <-- per-token slashes
      - quotes, spaces
      - list/tuple of ints/strings
    """
    # list/tuple of ints or digit-strings -> join
    if isinstance(x, (list, tuple)) and x and all(
        isinstance(v, int) or (isinstance(v, str) and v.strip().lstrip("/").isdigit())
        for v in x
    ):
        return ",".join(str(int(str(v).strip().lstrip("/"))) for v in x)

    s = str(x).strip()

    # strip wrappers/quotes
    s = s.strip('"').strip("'")
    s = s.replace(" ", "")

    # If it looks like weird nested tuples, reject clearly
    if s.startswith("[(") or s.startswith("(("):
        raise ValueError(f"goal_blocks invalid: expected 'a,b,c,...' got {x!r}")

    # split by comma, strip leading '/' from EACH token
    tokens = [t for t in s.split(",") if t != ""]
    cleaned = []
    for t in tokens:
        t2 = t.strip().lstrip("/")   # <-- key fix
        if not t2.isdigit():
            raise ValueError(f"goal_blocks invalid: expected 'a,b,c,...' got {x!r}")
        cleaned.append(t2)

    if not cleaned:
        raise ValueError(f"goal_blocks invalid: expected 'a,b,c,...' got {x!r}")

    return ",".join(cleaned)


In [4]:
from pydantic import BaseModel, Field

class ColorBlocksArgs(BaseModel):
    start_blocks: str = Field(..., description='EXACT "(a,b),(c,d),..." no spaces')
    goal_blocks: str  = Field(..., description='EXACT "g0,g1,g2,..." no spaces')


In [6]:
from mcp.server.fastmcp import FastMCP
import math

# Initialize FastMCP
mcp = FastMCP("Unified Solver")

@mcp.tool()
def calculate_sum(a: float, b: float) -> float:
    """Calculates the sum of two numbers."""
    return a + b

@mcp.tool()
def calculate_power(base: float, exponent: float) -> float:
    """Calculates the power of a base number."""
    return math.pow(base, exponent)

# TO DO: Add more tools as needed for your application


@mcp.tool()
def solve_cost_mcp(start_blocks, goal_blocks) -> float:
    """
    Compute the optimal solution cost for the Color Blocks search problem.

    INPUTS (MUST be single values, not nested objects):
    - start_blocks: string "(a,b),(c,d),..."  (each block is a pair top,bottom)
    - goal_blocks:  string "g0,g1,g2,..." (top colors required at each position)

    OUTPUT:
    - A single number (float): optimal path cost (each move costs 1).
    """
    start_blocks = normalize_start_blocks_any(start_blocks)
    goal_blocks  = normalize_goal_blocks_any(goal_blocks)

    init_goal_for_heuristics(goal_blocks)
    init_goal_for_search(goal_blocks)
    start_state = color_blocks_state(blocks_str=start_blocks)
    path = search(start_state, advanced_heuristic)
    return float("inf") if path is None else float(path[-1].g)

def solve_cost_wrapper(args: ColorBlocksArgs) -> float:
    return solve_cost_mcp(args.start_blocks, args.goal_blocks)


def as_langchain_tool_from_fn(fn, name: str, description: str, args_schema=None):
    return StructuredTool.from_function(
        func=fn,
        name=name,
        description=description,
        args_schema=args_schema,
        coroutine=fn if asyncio.iscoroutinefunction(fn) else None,
    )



solve_cost_tool = as_langchain_tool_from_fn(
    solve_cost_wrapper,
    name="solve_color_blocks_cost",
    description=(
        "Compute optimal solution cost for Color Blocks.\n"
        "REQUIRES BOTH: start_blocks and goal_blocks.\n"
        "Returns a single float cost."
    ),
    args_schema=ColorBlocksArgs
)




## 2. LLM + MCP

### 2.1. Global instance of our LLM

In [7]:
import os
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI


# Choose your model here, can be Ollama or Google Gemini
# Can also switch between different model sizes as needed
model = "qwen3:0.6b"
model = "ibm/granite4:350m"
global_llm = ChatOllama(model=model, temperature=0.0)


# SETUP API KEY if using Google Gemini
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY_HERE"

# model = "gemini-2.5-flash"
# model = "gemini-2.5-flash-lite"
# global_llm = ChatGoogleGenerativeAI(model=model, temperature=0)


### 2.2. Our agent graph

In [8]:
from langgraph.graph import MessagesState, START, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver # Optional: For saving graph state


def create_agent_graph(sys_msg, tools):
    """ Creates a LangGraph StateGraph with the given tools integrated."""

    llm = global_llm

    if tools:
        llm_with_tools = llm.bind_tools(tools)
    else:
        llm_with_tools = llm

    # Node
    def assistant(state: MessagesState):
        return {
            "messages": [
                llm_with_tools.invoke([sys_msg] + state["messages"], think=False)
            ]
        }

    # Graph
    builder = StateGraph(MessagesState)

    # Define the basic graph structure
    builder.add_node("assistant", assistant)
    builder.add_edge(START, "assistant")

    if tools:
        builder.add_node("tools", ToolNode(tools))  
        builder.add_conditional_edges(
            "assistant",
            tools_condition,
        )
        builder.add_edge("tools", "assistant")

    react_graph = builder.compile()

    return react_graph


async def run_agent(prompt, tools, sys_msg=""):

    sys_msg = SystemMessage(content=sys_msg)

    # 3. Create Graph
    graph = create_agent_graph(sys_msg, tools)
    
    # 4. Run (using ainvoke for async tools)
    config = {"configurable": {"thread_id": "1"}}
    result = await graph.ainvoke({"messages": [HumanMessage(content=prompt)]}, config)

    last_msg = result["messages"][-1].content

    # Extract tool names and outputs
    tools_used = []
    tools_output = []
    
    # Parsing logic specific to your request
    for msg in result["messages"]:
        # In LangChain, tool calls are usually in 'tool_calls' attribute of AIMessage
        # or 'name' attribute if it is a ToolMessage
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
             for tool_call in msg.tool_calls:
                tools_used.append(tool_call['name'])
        
        if msg.type == 'tool':
            tools_output.append(msg.content)

    return last_msg, tools_used, tools_output

### 2.3. Tools that run spacific agent (with tools and without)

In [9]:

from langchain_core.tools import StructuredTool
import asyncio

WITH_TOOLS_SYS = """
You are the WITH-TOOLS assistant.

You MUST call the tool solve_cost_mcp EXACTLY ONCE using the given arguments.

Input format (use EXACTLY as provided):
- start_blocks: a single string in the format "(a,b),(c,d),..."
- goal_blocks: a single string in the format "x,y,z,..."

Rules:
- Do NOT add or remove characters.
- Do NOT add leading symbols (/, [, ], quotes).
- Do NOT split start_blocks into a list.
- Do NOT change the order of blocks.

After the tool returns:
- Output ONLY the numeric solution cost.
- Do NOT include explanations or extra text.
"""

NO_TOOLS_SYS = """
You are the NO-TOOLS assistant.

You are solving a SEARCH problem WITHOUT tools and WITHOUT running code.
You must still produce an answer.

Domain rules:
- State is a list of pairs (top,bottom).
- spin(i): swaps top and bottom of block i. Cost = 1.
- flip(i): reverses the order of blocks from index i to the end. Cost = 1.
- Goal: for each position i, the top color equals goal_blocks[i].

Instructions:
- Reason mentally using the rules above.
- If unsure, prefer a CONSERVATIVE estimate (slightly higher, not lower).
- Never refuse and never ask for more information.

Output:
Output:
- Output ONLY a single number.
- NO explanation, NO steps, NO extra text.
"""


@mcp.tool()
async def ask_agent_with_tools(start_blocks: str, goal_blocks: str) -> str:
    """
    WITH-TOOLS assistant.
    Must call solve_cost_mcp exactly once and return ONLY the numeric cost.
    """

    sys_msg = f"""{WITH_TOOLS_SYS}

Use these exact inputs:
start_blocks = "{start_blocks}"
goal_blocks  = "{goal_blocks}"
"""
    # Only the solver tool is available
    tools = [solve_cost_mcp]

    last_msg, _, _ = await run_agent(
        prompt="Compute the solution cost.",
        tools=tools,
        sys_msg=sys_msg
    )

    return last_msg

@mcp.tool()
async def ask_agent_without_tools(start_blocks: str, goal_blocks: str) -> str:
    """
    NO-TOOLS assistant.
    Must estimate and return ONLY a numeric cost.
    """

    sys_msg = f"""{NO_TOOLS_SYS}

Start blocks: {start_blocks}
Goal blocks: {goal_blocks}
"""

    last_msg, _, _ = await run_agent(
        prompt="Estimate the minimal solution cost. Output only a number.",
        tools=[],
        sys_msg=sys_msg
    )

    return last_msg


def as_langchain_tool_from_fn(fn, name: str, description: str, args_schema):
    return StructuredTool.from_function(
        func=fn,
        name=name,
        description=description,
        args_schema=args_schema,
        coroutine=fn if asyncio.iscoroutinefunction(fn) else None,
    )


async def ask_with_tools_wrapper(
    start_blocks: str = None,
    goal_blocks: str = None,
    args: ColorBlocksArgs = None,
    **kwargs
) -> str:
    if args is not None:
        start_blocks = start_blocks or args.start_blocks
        goal_blocks = goal_blocks or args.goal_blocks

    if start_blocks is None or goal_blocks is None:
        raise ValueError(f"Missing args: start_blocks={start_blocks}, goal_blocks={goal_blocks}")

    return await ask_agent_with_tools(start_blocks, goal_blocks)


async def ask_without_tools_wrapper(
    start_blocks: str = None,
    goal_blocks: str = None,
    args: ColorBlocksArgs = None,
    **kwargs
) -> str:
    if args is not None:
        start_blocks = start_blocks or args.start_blocks
        goal_blocks = goal_blocks or args.goal_blocks

    if start_blocks is None or goal_blocks is None:
        raise ValueError(f"Missing args: start_blocks={start_blocks}, goal_blocks={goal_blocks}")

    return await ask_agent_without_tools(start_blocks, goal_blocks)


ask_with_tools_tool = as_langchain_tool_from_fn(
    ask_with_tools_wrapper,
    name="ask_agent_with_tools",
    description=(
        "Calls the WITH-TOOLS assistant.\n"
        "REQUIRES BOTH start_blocks and goal_blocks.\n"
        "Returns ONLY a numeric cost as a string."
    ),
    args_schema=ColorBlocksArgs
)

ask_without_tools_tool = as_langchain_tool_from_fn(
    ask_without_tools_wrapper,
    name="ask_agent_without_tools",
    description=(
        "Calls the NO-TOOLS assistant.\n"
        "REQUIRES BOTH start_blocks and goal_blocks.\n"
        "Returns ONLY a numeric cost as a string."
    ),
    args_schema=ColorBlocksArgs
)





## 3. Run the Test

In [12]:
sys_msg = """
You are a JUDGE agent implemented using LangGraph.

Your role is to receive the outputs of TWO previous agents:
1) A WITH-TOOLS agent
2) A NO-TOOLS agent

You MUST run both agents, analyze their outputs, explain the differences
between their solutions, and then write a short summary.

====================
MANDATORY TOOL CALLS
====================
You MUST do the following in order:

1) Call ask_agent_with_tools(start_blocks, goal_blocks) EXACTLY ONCE.
2) Call ask_agent_without_tools(start_blocks, goal_blocks) EXACTLY ONCE.

Do NOT skip any call.
Do NOT retry tools.

====================
ANALYSIS TASK
====================
After receiving BOTH outputs:

- Compare the two solutions.
- Explain the differences between them, such as:
  * Use of tools vs. reasoning without tools
  * Reliability and optimality of the solution
  * Clarity and correctness of the result
- Do NOT recompute the solution yourself.
- Base your explanation ONLY on the agents’ outputs.

====================
OUTPUT REQUIREMENTS
====================
Write a SHORT, CLEAR explanation in plain English that includes:

1) A comparison of the two agents’ solutions
2) An explanation of the key differences
3) A brief concluding summary

You are NOT required to output numeric costs or choose a winner unless it
naturally follows from your explanation.

====================
PROHIBITED
====================
- Do NOT solve the puzzle yourself
- Do NOT invent results
- Do NOT ignore either agent’s output
- Do NOT call any tool more than once
"""




test_cases = [
    ("(5,2),(1,3),(9,22),(21,4)", "2,22,4,3"),
    ("(1,2),(3,4),(5,6),(7,8)", "7,5,3,1"),
    ("(1,2),(3,4),(5,6),(7,8)", "3,7,1,5"),
    ("(10,1),(2,20),(3,30),(4,40)", "4,10,3,2"),
    ("(8,9),(7,6),(5,4),(3,2)", "7,3,8,5"),
]

for i, (start_blocks, goal_blocks) in enumerate(test_cases, 1):
    print(f"\n================= CASE {i} =================")

    judge_prompt = f"""
Instance:
start_blocks = "{start_blocks}"
goal_blocks  = "{goal_blocks}"
"""

    tool_list = [ask_with_tools_tool, ask_without_tools_tool]
    response, tools_used, outputs = await run_agent(judge_prompt, tool_list, sys_msg)

    print("=== RESPONSE ===")
    print(response)
    print("=== TOOLS USED ===")
    print(tools_used)
    print("=== TOOL OUTPUTS ===")
    print(outputs)




================= CASE 1 =================


HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== RESPONSE ===
The WITH-TOOLS solution cost is 6.0.
The NO-TOOLS solution cost is 1.
=== TOOLS USED ===
['ask_agent_with_tools', 'ask_agent_without_tools']
=== TOOL OUTPUTS ===
['The optimal solution cost is 6.0.', '1']

================= CASE 2 =================


HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== RESPONSE ===
The WITH-TOOLS solution cost is 1.0. The NO-TOOLS solution also costs 1.0. Both approaches yield the same optimal cost of 1.0.
=== TOOLS USED ===
['ask_agent_with_tools', 'ask_agent_without_tools']
=== TOOL OUTPUTS ===
['The optimal solution cost is 1.0.', '1']

================= CASE 3 =================


...

=== RESPONSE ===
The WITH-TOOLS solution cost is 4.0.
The NO-TOOLS solution cost is 1.0.
=== TOOLS USED ===
['ask_agent_with_tools', 'ask_agent_without_tools']
=== TOOL OUTPUTS ===
['The solution cost is 4.0.', '1']

================= CASE 4 =================


HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== RESPONSE ===
The WITH-TOOLS solution cost is 3.0.
The NO-TOOLS solution cost is 1.0.
=== TOOLS USED ===
['ask_agent_with_tools', 'ask_agent_without_tools']
=== TOOL OUTPUTS ===
['The optimal solution cost is 3.0.', '1']

================= CASE 5 =================


HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== RESPONSE ===
The WITH-TOOLS solution cost is 4.0.
The NO-TOOLS solution cost is 1.
=== TOOLS USED ===
['ask_agent_with_tools', 'ask_agent_without_tools']
=== TOOL OUTPUTS ===
['The solution cost is 4.0.', '1']
